In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from keras.models import Model
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import confusion_matrix,classification_report

def load_file(filepath):
    dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values



# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    loaded = np.dstack(loaded)
    return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y


# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + '..//Datasets//UCI HAR Dataset//')
    # load all test
    testX, testy = load_dataset_group('test', prefix + '..//Datasets//UCI HAR Dataset//')
    
    #zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    #one hot encode y
    trainy_one_hot = to_categorical(trainy)
    testy_one_hot = to_categorical(testy)
    print(trainX.shape, trainy.shape, trainy_one_hot.shape, testX.shape, testy.shape, testy_one_hot.shape)
    return trainX, trainy, trainy_one_hot, testX, testy, testy_one_hot


In [2]:
trainX, trainy, trainy_one_hot, testX, testy, testy_one_hot = load_dataset()

(7352, 128, 9) (7352, 1) (7352, 6) (2947, 128, 9) (2947, 1) (2947, 6)


In [3]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train_one_hot,y_val_one_hot,y_train,y_val=train_test_split(trainX, trainy_one_hot, trainy,test_size=0.3,random_state=100)

In [4]:
X_train.shape,X_val.shape,y_train_one_hot.shape,y_val_one_hot.shape,y_train.shape,y_val.shape

((5146, 128, 9), (2206, 128, 9), (5146, 6), (2206, 6), (5146, 1), (2206, 1))

In [5]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_one_hot.shape[1]

In [6]:
n_timesteps, n_features, n_outputs 

(128, 9, 6)

In [7]:
sys.path.append("..//Base_Models")

In [8]:
from model1 import cnn

In [9]:
m1=cnn(n_timesteps, n_features, n_outputs)

In [10]:
model1=m1.do_compile(X_train, testX,y_train_one_hot,testy_one_hot)

Epoch 1/30
81/81 [==============================] - 32s 301ms/step - loss: 0.3540 - accuracy: 0.8599 - val_loss: 0.2843 - val_accuracy: 0.8996
Epoch 2/30
81/81 [==============================] - 9s 117ms/step - loss: 0.1277 - accuracy: 0.9450 - val_loss: 0.2060 - val_accuracy: 0.9121
Epoch 3/30
81/81 [==============================] - 9s 110ms/step - loss: 0.1119 - accuracy: 0.9526 - val_loss: 0.2272 - val_accuracy: 0.9209
Epoch 4/30
81/81 [==============================] - 8s 104ms/step - loss: 0.1048 - accuracy: 0.9536 - val_loss: 0.2041 - val_accuracy: 0.9270
Epoch 5/30
81/81 [==============================] - 8s 101ms/step - loss: 0.0978 - accuracy: 0.9567 - val_loss: 0.2281 - val_accuracy: 0.9128
Epoch 6/30
81/81 [==============================] - 8s 96ms/step - loss: 0.0948 - accuracy: 0.9574 - val_loss: 0.1915 - val_accuracy: 0.9216
Epoch 7/30
81/81 [==============================] - 8s 96ms/step - loss: 0.0909 - accuracy: 0.9623 - val_loss: 0.2187 - val_accuracy: 0.9165
Epoch 8

In [11]:
predy=m1.prediction(testX)
cm=confusion_matrix(testy,predy)
print(cm)
print(acc(testy,predy))
print("classification_report of model1:")
print(classification_report(testy,predy))
sns.heatmap(cm, annot=True)

[[488   0   2   5   0   1]
 [ 13 434   1  23   0   0]
 [  0   2 413   4   0   1]
 [  0   1   0 416  73   1]
 [  1   0   0  69 462   0]
 [  0  20   0   0   0 517]]
0.9263657957244655
classification_report of model1:
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       496
           1       0.95      0.92      0.94       471
           2       0.99      0.98      0.99       420
           3       0.80      0.85      0.83       491
           4       0.86      0.87      0.87       532
           5       0.99      0.96      0.98       537

    accuracy                           0.93      2947
   macro avg       0.93      0.93      0.93      2947
weighted avg       0.93      0.93      0.93      2947



<AxesSubplot:>

In [24]:
model1.save("Models_h5//model1.h5")

In [11]:
from model2 import cnn_lstm

In [12]:
m2=cnn_lstm(n_timesteps, n_features, n_outputs)

In [13]:
model2=m2.do_compile(X_train, testX,y_train_one_hot,testy_one_hot)

Epoch 1/30
81/81 [==============================] - 14s 122ms/step - loss: 0.4875 - accuracy: 0.8070 - val_loss: 0.2566 - val_accuracy: 0.8945
Epoch 2/30
81/81 [==============================] - 9s 112ms/step - loss: 0.1503 - accuracy: 0.9435 - val_loss: 0.2155 - val_accuracy: 0.9128
Epoch 3/30
81/81 [==============================] - 9s 113ms/step - loss: 0.1345 - accuracy: 0.9450 - val_loss: 0.2274 - val_accuracy: 0.9094
Epoch 4/30
81/81 [==============================] - 9s 114ms/step - loss: 0.1208 - accuracy: 0.9454 - val_loss: 0.2012 - val_accuracy: 0.9091
Epoch 5/30
81/81 [==============================] - 9s 114ms/step - loss: 0.1075 - accuracy: 0.9539 - val_loss: 0.2278 - val_accuracy: 0.9077
Epoch 6/30
81/81 [==============================] - 9s 110ms/step - loss: 0.1066 - accuracy: 0.9536 - val_loss: 0.2171 - val_accuracy: 0.9196
Epoch 7/30
81/81 [==============================] - 9s 111ms/step - loss: 0.0994 - accuracy: 0.9539 - val_loss: 0.2407 - val_accuracy: 0.9131
Epoch

In [18]:
predy=m2.prediction(testX)
cm=confusion_matrix(testy,predy)
print(cm)
print(acc(testy,predy))
print("classification_report of model2:")
print(classification_report(testy,predy))
sns.heatmap(cm, annot=True)

[[472   7  17   0   0   0]
 [  0 463   8   0   0   0]
 [  0   1 419   0   0   0]
 [  0   2   0 400  82   7]
 [  0   0   0  40 492   0]
 [  0   0   0   0   0 537]]
0.9443501866304717
classification_report of model2:
              precision    recall  f1-score   support

           0       1.00      0.95      0.98       496
           1       0.98      0.98      0.98       471
           2       0.94      1.00      0.97       420
           3       0.91      0.81      0.86       491
           4       0.86      0.92      0.89       532
           5       0.99      1.00      0.99       537

    accuracy                           0.94      2947
   macro avg       0.95      0.95      0.94      2947
weighted avg       0.95      0.94      0.94      2947



<AxesSubplot:>

In [19]:
model2.save("Models_h5//model2.h5")


In [14]:
model2.save("Models_h5//model2_2.h5")

In [16]:
from model3 import Conv_LSTM

In [17]:
m3=Conv_LSTM(n_timesteps, n_features, n_outputs)

In [19]:
model3=m3.do_compile(X_train,testX,y_train_one_hot,testy_one_hot)

Trial 10 Complete [00h 01m 15s]
val_accuracy: 0.9165253043174744

Best val_accuracy So Far: 0.9243298172950745
Total elapsed time: 00h 13m 31s
INFO:tensorflow:Oracle triggered exit


Epoch 11/30
161/161 [==============================] - 12s 41ms/step - loss: 0.1452 - accuracy: 0.9425 - val_loss: 0.2302 - val_accuracy: 0.9260
Epoch 12/30
161/161 [==============================] - 5s 34ms/step - loss: 0.1085 - accuracy: 0.9522 - val_loss: 0.3375 - val_accuracy: 0.8982
Epoch 13/30
161/161 [==============================] - 5s 34ms/step - loss: 0.1452 - accuracy: 0.9440 - val_loss: 0.4125 - val_accuracy: 0.9097
Epoch 14/30
161/161 [==============================] - 5s 34ms/step - loss: 0.1191 - accuracy: 0.9522 - val_loss: 0.2907 - val_accuracy: 0.9206
Epoch 15/30
161/161 [==============================] - 6s 35ms/step - loss: 0.1081 - accuracy: 0.9522 - val_loss: 0.3346 - val_accuracy: 0.9158
Epoch 16/30
161/161 [==============================] - 6s 35ms/step - loss: 0.1234 - accuracy: 0.9539 - val_loss: 0.3607 - val_accuracy: 0.9121
Epoch 17/30
161/161 [==============================] - 5s 34ms/step - loss: 0.1101 - accuracy: 0.9545 - val_loss: 0.3890 - val_accuracy

In [20]:
predy=m3.prediction(testX)
cm=confusion_matrix(testy,predy)
print(cm)
print(acc(testy,predy))
print("classification_report of model3:")
print(classification_report(testy,predy))

[[460   2   2   0  32   0]
 [  2 460   9   0   0   0]
 [  0   3 417   0   0   0]
 [  0  11   0 410  70   0]
 [  0   0   0  89 443   0]
 [  0   0   0   0   0 537]]
0.9253478113335596
classification_report of model3:
              precision    recall  f1-score   support

           0       1.00      0.93      0.96       496
           1       0.97      0.98      0.97       471
           2       0.97      0.99      0.98       420
           3       0.82      0.84      0.83       491
           4       0.81      0.83      0.82       532
           5       1.00      1.00      1.00       537

    accuracy                           0.93      2947
   macro avg       0.93      0.93      0.93      2947
weighted avg       0.93      0.93      0.93      2947



In [13]:
model3.save("Models_h5//model3.h5")

In [21]:
model3.save("Models_h5//model3_2.h5")

In [8]:
from model4 import Layered_LSTM

In [9]:
m4=Layered_LSTM(n_timesteps, n_features, n_outputs)

In [10]:
model4=m4.do_compile(X_train, testX,y_train_one_hot,testy_one_hot)

Epoch 1/30
81/81 [==============================] - 16s 65ms/step - loss: 0.7300 - accuracy: 0.7217 - val_loss: 1.6820 - val_accuracy: 0.3946
Epoch 2/30
81/81 [==============================] - 4s 50ms/step - loss: 0.2711 - accuracy: 0.9024 - val_loss: 2.7613 - val_accuracy: 0.4201
Epoch 3/30
81/81 [==============================] - 4s 50ms/step - loss: 0.2014 - accuracy: 0.9273 - val_loss: 1.5058 - val_accuracy: 0.5891
Epoch 4/30
81/81 [==============================] - 4s 50ms/step - loss: 0.1504 - accuracy: 0.9423 - val_loss: 1.5167 - val_accuracy: 0.6817
Epoch 5/30
81/81 [==============================] - 4s 50ms/step - loss: 0.1691 - accuracy: 0.9370 - val_loss: 1.2652 - val_accuracy: 0.7265
Epoch 6/30
81/81 [==============================] - 4s 50ms/step - loss: 0.1541 - accuracy: 0.9419 - val_loss: 0.4966 - val_accuracy: 0.8680
Epoch 7/30
81/81 [==============================] - 4s 50ms/step - loss: 0.1622 - accuracy: 0.9411 - val_loss: 0.3253 - val_accuracy: 0.8948
Epoch 8/30
8

In [11]:
predy=m4.prediction(testX)
cm=confusion_matrix(testy,predy)
print(cm)
print(acc(testy,predy))
print("classification_report of model4:")
print(classification_report(testy,predy))

[[488   3   4   0   1   0]
 [  1 450   4  16   0   0]
 [  4   5 411   0   0   0]
 [  0  10   0 343 137   1]
 [  1   0   0  44 487   0]
 [  0   0   0   0   0 537]]
0.9216152019002375
classification_report of model4:
              precision    recall  f1-score   support

           0       0.99      0.98      0.99       496
           1       0.96      0.96      0.96       471
           2       0.98      0.98      0.98       420
           3       0.85      0.70      0.77       491
           4       0.78      0.92      0.84       532
           5       1.00      1.00      1.00       537

    accuracy                           0.92      2947
   macro avg       0.93      0.92      0.92      2947
weighted avg       0.92      0.92      0.92      2947



In [22]:
model4.save("Models_h5//model4.h5")

In [ ]:
model4.save("Models_h5//model4_2.h5")